In [1]:
import numpy as np
import pandas as pd
import torch
import matchzoo as mz
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from rank_bm25 import BM25Okapi
from multiprocessing import Pool,Lock
import tqdm
import catboost
import operator
from sklearn.feature_extraction.text import TfidfVectorizer
import lightgbm as lg
from gensim.models import FastText

In [2]:
#data 
queries = {}
with open('./norm_q.txt') as fin:
    lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        if line[0] == '':
            line.pop(0)
        queries[line[0]] = line[1]

        titles= {}
with open("./norm_tits.txt" ,'r', encoding='utf-8') as fin:
    for line in fin.readlines():
        line=line.split('\t')
        titles[line[0]]= line[1][:-1]

train=[]
train_titles = []
train_queries = []
trq = set()
with open("./train.marks.tsv/train.marks.tsv", 'r') as fin:
    for line in fin.readlines():
        line=line[:-1].split('\t')
        if line[1] in titles.keys():
            train.append([line[0],line[1], line[2]])
            train_titles.append([line[1], titles[line[1]]])
            if line[0] not in trq:
                train_queries.append([line[0], queries[line[0]]])
                trq.add(line[0])

                
test=[]
test_titles = []
test_queries = []
teq = set()
with open("./sample.csv/sample.csv", 'r') as fin:
    fin.readline()
    for line in fin.readlines():
        line=line[:-1].split(',')
        if line[1] in titles.keys():
            test.append([line[0],line[1],-1])
            test_titles.append([line[1], titles[line[1]]])
            if line[0] not in teq:
                test_queries.append([line[0], queries[line[0]]])
                teq.add(line[0])

alq = list(train) + list(test)
dctqs = {}
for q, t, n in alq:
    if q not in dctqs.keys():
        dctqs[q] = []
    dctqs[q].append(t)
alqueries = train_queries + test_queries
id_q = {}
for el in alqueries:
    id_q[el[0]] = el[1]   

altits = {}
for el in train_titles + test_titles:
    altits[el[0]] = el[1]

In [9]:
# large USE
import tensorflow as tf
from absl import logging
import sentencepiece
import tensorflow_text
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns


embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

def cos(sp1,sp2):
    #sp1 = sp1.todense()
    #sp2 = sp2.todense()
    return np.float64(np.dot(sp1,sp2.T)/np.linalg.norm(sp1)/np.linalg.norm(sp2))

import time
def getsimuse(qid):
    raw_tits = {}
    global id_q
    global altits
    global embed
    #t1 = time.time()
    for dc in dctqs[qid]:
        try:
            raw_tits[dc] = altits[dc]
        except:
            print(qid, dc)
    q = embed([id_q[qid]]).numpy()
    embs = embed(list(raw_tits.values())).numpy()
    #t2 = time.time()
    #print(t2 - t1)
    results = []
    with open('./luse/{}.txt'.format(qid),'w') as fout:
        for i,doc in enumerate(raw_tits.keys()):
            fout.write(str(qid) + '\t' +str(doc) + '\t' + str(cos(q,embs[i])) + '\n')
    #print(time.time() - t2)
    del embs
    del q
    del raw_tits
    return results
for q in tqdm.tqdm(dctqs.keys()):
    r = getsimuse(q)
lfiles = os.listdir(outdata)
with open('luse.txt','w') as fout:
    for file in lfiles:
        with open(outdata+file) as fin:
            line = fin.readline()
            while line != '':
                fout.write(line)
                line = fin.readline()

In [7]:
dtitles = [TaggedDocument(doc, [i]) for i, doc in train_titles]
dqueries = [TaggedDocument(doc, [i]) for i, doc in train_queries]
dstitles = [TaggedDocument(doc, [i]) for i, doc in test_titles]
dsqueries = [TaggedDocument(doc, [i]) for i, doc in test_queries]


In [7]:
#docvec similarity on queries
from nltk.tokenize import word_tokenize

In [9]:
aq = [TaggedDocument(doc,[i]) for i, doc in alqueries]

In [10]:
dv = Doc2Vec(aq, vector_size=128, window=2, min_count=1, workers=4)

In [10]:
max_epochs = 100
alpha = 0.0025
dv = Doc2Vec(dm=0, vector_size=100, negative=5, hs=0, min_count=2, sample = 0)
dv.build_vocab(aq)
for epoch in range(max_epochs):
    dv.train(aq,
    total_examples=dv.corpus_count,
    epochs=dv.iter)
    # decrease the learning rate
    dv.alpha -= 0.0002
    # fix the learning rate, no decay
    dv.min_alpha = dv.alpha

/home/pe/.conda/envs/pycode/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


In [11]:
dvecs = {}
for el in alqueries:
    dvecs[el[0]] = dv.docvecs[el[0]]

In [12]:
def cos(sp1,sp2):
    #sp1 = sp1.todense()
    #sp2 = sp2.todense()
    return np.float64(np.dot(sp1,sp2.T)/np.linalg.norm(sp1)/np.linalg.norm(sp2))

In [13]:
simdoc = {}
def getsim(key):
    global dvecs
    tmp = {}
    for tkey in dvecs.keys():
        if tkey != key:
            tmp[tkey] = cos(dvecs[key],dvecs[tkey])
    sortres = sorted(tmp.items(), key = operator.itemgetter(1), reverse = True)
    simdoc[key] = []
    res = []
    for i in range(20):
        res.append(sortres[i][0])
    return [key,res]
p = Pool(4)
sims = p.map(getsim, [key for key in dvecs.keys()])

In [14]:
with open('oldsimdoc.txt','w') as fout:
    for res in sims:
        fout.write(res[0]+'\t')
        for el in res[1]:
            fout.write(el+' ')
        fout.write('\n')

In [10]:
#qa USE
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tensorflow_text
module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3')
outdata = './quse/'
try:
    os.mkdir(outdata)
except:
    n = None
def getsimqa(qid):
    global id_q
    global altits
    global module
    global dctqs
    qe = module.signatures['question_encoder'](tf.constant(id_q))['outputs'].numpy()
    with open(outdata+qid+'.txt','w') as fout:
        for doc in dctqs.keys():
            try:
                title = altits[doc]
                with open('./data/{}.txt'.format(doc)) as fin:
                    text = fin.readline()
                re = module.signatures['response_embeddings'](input = tf.constant(title), context = tf.constant(text))['outputs'].numpy()
                fout.write(qid+'\t' + doc + '\t' + str(cos(qe,re)) + '\n')
                del title
                del text
                del re
            except:
                continue
for q in tqdm.tqdm(list(dctqs.keys())):
    getsimqa(q)
qfiles = os.listdir(outdata)
with open('quse.txt','w') as fout:
    for file in qfiles:
        with open(outdata+file) as fin:
            line = fin.readline()
            while line != '':
                fout.write(line)
                line = fin.readline()

In [12]:
#multiliggual USE
import os
import tensorflow as tf
from absl import logging
import sentencepiece
import tensorflow_text
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns


embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

def cos(sp1,sp2):
    #sp1 = sp1.todense()
    #sp2 = sp2.todense()
    return np.float64(np.dot(sp1,sp2.T)/np.linalg.norm(sp1)/np.linalg.norm(sp2))

import time
def getsimuse(qid):
    raw_tits = {}
    global id_q
    global altits
    global embed
    #t1 = time.time()
    for dc in dctqs[qid]:
        try:
            raw_tits[dc] = altits[dc]
        except:
            print(qid, dc)
    q = embed([id_q[qid]]).numpy()
    embs = embed(list(raw_tits.values())).numpy()
    #t2 = time.time()
    #print(t2 - t1)
    results = []
    with open('./muse/{}.txt'.format(qid),'w') as fout:
        for i,doc in enumerate(raw_tits.keys()):
            fout.write(str(qid) + '\t' +str(doc) + '\t' + str(cos(q,embs[i])) + '\n')
    #print(time.time() - t2)
    del embs
    del q
    del raw_tits
    return results
for q in tqdm.tqdm(dctqs.keys()):
    r = getsimuse(q)
mfiles = os.listdir('./muse/')
with open('muse.txt','w') as fout:
    for file in mfiles:
        with open('./muse/muse/'+file) as fin:
            line = fin.readline()
            while line != '':
                fout.write(line)
                line = fin.readline()

In [20]:
#docvec similarity on titles
from multiprocessing import Pool
def docdst(key):
    lst = []
    global dctqs
    global altits
    global id_q
    for el in dctqs[key]:
        lst.append([el,altits[el]])
    td = [TaggedDocument(doc,[i]) for i, doc in lst]
    td.append(TaggedDocument(id_q[key],[1111111]))
    dv = Doc2Vec(dm=0, vector_size=100, negative=5, hs=0, min_count=2, sample = 0)
    dv.build_vocab(td)
    for epoch in range(max_epochs):
        dv.train(tagged_data,
        total_examples=dv.corpus_count,
        epochs=dv.iter)
        dv.alpha -= 0.0002
        dv.min_alpha = dv.alpha
    with open('./dv/{}.txt'.format(key),'w') as fout:
        for el in dctqs[key]:
            fout.write(key + '\t' + el + '\t' + str(cos(dv.docvecs[1111111],dv.docvecs[el]))+'\n')
    return key
p = Pool(6)
p.map(docdst,[key for key in dctqs.keys()])

In [18]:
#fasttext trained
import fasttext.util
import fasttext
from gensim.models import FastText
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
model = FastText(min_count=1, size = 10, workers=8, negative=5, iter=10, min_n=2, max_n=10)
def getsim(q):
    global dctqs
    global id_q
    global altits
    raw = {}
    model = FastText(min_count=1, size = 10, workers=2, negative=3, iter=10, min_n=2, max_n=9)
    for doc in dctqs[q]:
        try:
            raw[doc] = word_tokenize(altits[doc])
        except:
            continue
    raw['-1'] = word_tokenize(id_q[q])
    model.build_vocab(raw.values())
    model.train(raw.values(), total_examples=model.corpus_count, epochs=10)
    with open('./trft/{}.txt'.format(q),'w') as fout:
        for outdoc in raw.keys():
            if outdoc != '-1':
                fout.write(str(outdoc)+ '\t'+str(cos(model.wv[altits[outdoc].replace(' ','_')], model.vw[id_q[q]].replace(' ','_'))) + '\n')
    return
p = Pool(4)
sims = p.map(getsim, [key for key in dctqs.keys()])

In [19]:
#fasttext wikipedia
fasttext.util.download_model('ru', if_exists='ignore')
ft = fasttext.load_model('cc.ru.300.bin')
def getsim(q):
    global dctqs
    global id_q
    global altits
    global ft
    raw = {}
    for doc in dctqs[q]:
        try:
            raw[doc] = ft.get_word_vector(altits[doc])
        except:
            continue
    raw['-1'] = ft.get_word_vector(id_q[q])
    with open('./ftsim/{}.txt'.format(q),'w') as fout:
        for outdoc in raw.keys():
            if outdoc != '-1':
                fout.write(str(outdoc)+ '\t'+str(cos(raw[outdoc],raw['-1'])) + '\n')
    return
p = Pool(4)
sims = p.map(getsim, [key for key in dctqs.keys()])